# Reducing Failure-Inducing Inputs

By construction, fuzzers create inputs that may be hard to read.  This causes issues during _debugging_, when a human has to analyze the exact cause of the failure.  In this chapter, we present techniques that _automatically reduce and simplify failure-inducing inputs to a minimum_ in order to ease debugging.

**Prerequisites**

* As reduction is typically used together with fuzzing, reading the [chapter on basic fuzzing](Fuzzer.ipynb) is a good idea.
* The simple "delta debugging" technique for reduction has no specific prerequisites.
* The later grammar-based techniques require knowledge on [derivation trees](GrammarFuzzer.ipynb) and [parsing](Parser.ipynb).

## Why Reducing?

At this point, we have seen a number of test generation techniques that all in some form produce input in order to trigger failures.  If they are successful – that is, the program actually fails – we must find out why the failure occurred and how to fix it.

Here's an example of such a situation.  We have a class `MysteryRunner` class with a `run()` method that – given its code – can occasionally fail.  But under which circumstances does this actually happen?  We have deliberately obscured the exact condition in order to make this non-obvious.

In [ ]:
import fuzzingbook_utils

In [ ]:
from Fuzzer import RandomFuzzer, Runner

In [ ]:
class MysteryRunner(Runner):
    def run(self, inp):
        x = inp.find(chr(0o17 + 0o31))
        y = inp.find(chr(0o27 + 0o22))
        if x >= 0 and y >= 0 and x < y:
            return (inp, Runner.FAIL)
        else:
            return (inp, Runner.PASS)

Let us fuzz the function until we find a failing input.

In [ ]:
mystery = MysteryRunner()
random_fuzzer = RandomFuzzer()
while True:
    inp = random_fuzzer.fuzz()
    result, outcome = mystery.run(inp)
    if outcome == mystery.FAIL:
        break

In [ ]:
failing_input = result
failing_input

Something in this input causes `MysteryRunner` to fail.  But what is it?

## Manual Input Reduction

One important step in the debugging process is _reduction_ – that is, to identify those circumstances of a failure that are relevant for the failure to occur, and to _omit_ (if possible) those parts that are not.  As Kernighan and Pike \cite{Kernighan1999} put it:

> For every circumstance of the problem, check whether it is relevant for the problem to occur.  If it is not, remove it from the problem report or the test case in question.

Specifically for inputs, they suggest a _divide and conquer_ process:

> Proceed by binary search.  Throw away half the input and see if the output is still wrong; if not, go back to the previous state and discard the other half of the input.

This is something we can easily try out.  For instance, we can see whether the error still occurs if we only feed in the first half:

In [ ]:
half_length = len(failing_input) // 2   # // is integer division
first_half = failing_input[:half_length]
mystery.run(first_half)

Nope – the first half alone does not suffice.  Maybe the second half?

In [ ]:
second_half = failing_input[half_length:]
mystery.run(second_half)

This did not go so well either.  We may still proceed by cutting away _smaller chunks_ – say, one character after another.  If our test is deterministic and easily repeated, it is clear that this process eventually will yield a reduced input.  But still, it is a rather inefficient process, especially for long inputs.  What we need is a _strategy_ that effectively minimizes a failure-inducing input – a strategy that can be automated.

## Delta Debugging

One strategy to effectively reduce failure-inducing inputs is _delta debugging_ \cite{Zeller2002}.  Delta Debugging implements the "binary search" strategy, as listed above, but with a twist: If neither half fails (also as above), it keeps on cutting away smaller and smaller chunks from the input, until it eliminates individual characters.  Thus, after cutting away the first half, we cut away
the first quarter, the second quarter, and so on.

Let us illustrate this on our example, and see what happens if we cut away the first quarter.

In [ ]:
quarter_length = len(failing_input) // 4
input_without_first_quarter = failing_input[quarter_length:]
mystery.run(input_without_first_quarter)

Ah! This has failed, and reduced our failing input by 25%.  Let's remove another quarter.

In [ ]:
input_without_first_and_second_quarter = failing_input[quarter_length * 2:]
mystery.run(input_without_first_and_second_quarter)

This is not too surprising, as we had that one before:

In [ ]:
second_half

In [ ]:
input_without_first_and_second_quarter

How about removing the third quarter, then?

In [ ]:
input_without_first_and_third_quarter = failing_input[quarter_length:quarter_length * 2] + failing_input[quarter_length * 3:]
mystery.run(input_without_first_and_third_quarter)

How about removing the third quarter, then?

In [ ]:
input_without_first_and_fourth_quarter = failing_input[quarter_length:quarter_length * 3]
mystery.run(input_without_first_and_fourth_quarter)

Yes!  This has succeeded.  Our input is now 50% smaller.

We have now tried to remove pieces that make up $\frac{1}{2}$ and $\frac{1}{4}$ of the original failing string.  In the next iteration, we would go and remove even smaller pieces – $\frac{1}{8}$, $\frac{1}{16}$ and so on.  We continue until we are down to $\frac{1}{97}$ – that is, individual characters.

However, this is comething we happily let a computer do for us.  We first introduce a `Reducer` class as an abstract superclass for all kinds of reducers.  The `test()` methods runs a single test (with logging, if so wanted); the `reduce()` method will eventually reduce an input to the minimum.

In [ ]:
class Reducer(object):
    def __init__(self, runner, log=False):
        """Attach reducer to the given `runner`"""
        self.runner = runner
        self.log = log
        self.reset()

    def reset(self):
        self.tests = 0

    def test(self, inp):
        result, outcome = self.runner.run(inp)
        self.tests += 1
        if self.log:
            print("Test #%d" % self.tests, repr(inp), repr(len(inp)), outcome)
        return outcome

    def reduce(self, inp):
        self.reset()
        # Default: Don't reduce
        return inp

The `CachingReducer` variant saves test results, such that we don't have to run the same tests again and again:

In [ ]:
class CachingReducer(Reducer):
    def reset(self):
        super().reset()
        self.cache = {}

    def test(self, inp):
        if inp in self.cache:
            return self.cache[inp]
        
        outcome = super().test(inp)
        self.cache[inp] = outcome
        return outcome

Here comes the _Delta Debugging_ reducer.  Delta Debugging implements the strategy sketched above: It first removes larger chunks of size $\frac{1}{2}$; if this does not fail, then we proceed to chunks of size $\frac{1}{4}$, then $\frac{1}{8}$ and so on.

Our implementation uses almost the same Python code as Zeller in \cite{Zeller2011}; the only difference is that it has been adapted to work on Python 3 and our `Runner` framework.    The variable `n` (initially 2) indicates the granularity – in each step, chunks of size $\frac{1}{n}$ are cut away.  If none of the test fails (`some_complement_is_failing` is False), then `n` is doubled – until it reaches the length of the input.

In [ ]:
class DeltaDebuggingReducer(CachingReducer):
    def reduce(self, inp):
        self.reset()
        assert self.test(inp) == Runner.FAIL

        n = 2     # Initial granularity
        while len(inp) >= 2:
            start = 0
            subset_length = len(inp) / n
            some_complement_is_failing = False

            while start < len(inp):
                complement = inp[:int(start)] + inp[int(start + subset_length):]

                if self.test(complement) == Runner.FAIL:
                    inp = complement
                    n = max(n - 1, 2)
                    some_complement_is_failing = True
                    break

                start += subset_length

            if not some_complement_is_failing:
                if n == len(inp):
                    break
                n = min(n * 2, len(inp))

        return inp

To see how the `DeltaDebuggingReducer` works, let us run it on our failing input.  With each step, we see how the remaining input gets smaller and smaller, until only two characters remain:

In [ ]:
dd_reducer = DeltaDebuggingReducer(mystery, log=True)
dd_reducer.reduce(failing_input)

Now we know why `MysteryRunner` fails – it suffices that the input contains two matching parentheses.  Delta Debugging determines this is 29 steps.  Its result is _1-minimal_, meaning that every character contained is required to produce the error; removing any (as seen in tests `#27` and `#28`, above) no longer makes the test fail.  This property is guaranteed by the delta debugging algorithm, which in its last stage always tries to delete characters one by one.

A reduced test case such as the one above has many advantages:

* A reduced test case __reduces the _cognitive load_ of the programmer__.  The test case is shorter and focused, and thus does not burden the programmer with irrelevant details.  A reduced input typically leads to shorter executions and smaller program states, both of which reduce the search space as it comes to understanding the bug.  In our case, we have eliminated lots of irrelevant input – only the two characters the reduced input contains are relevant.

* A reduced test case __is easier to communicate__.  All one needs here is the summary: `MysteryRunner fails on "()"`, which is much better than `MysteryRunner fails on a 4100-character input (attached)`.

* A reduced test case helps in __identifying duplicates__.  If similar bugs have been reported already, and all of them have been reduced to the same cause (namely that the input contains matching parentheses), then it becomes obvious that all these bugs are different symptoms of the same underlying cause – and would all be resolved at once with one code fix.

How effective is delta debugging?  In the best case (wither the left half or the right half fails), the number of tests is logarithmic proportional to the length $n$ of an input (i.e., $O(\log_2 n)$); this is the same complexity as binary search.  In the worst case, though, delta debugging can require a number of tests proportional to $n^2$  (i.e., $O(n^2)$) – this happens in the case when we are down to character granularity, and we have to repeatedly tried to delete all characters, only to find that deleting the last character results in a failure \cite{Zeller2002}.  (This is a pretty pathological situation, though.)

In general, delta debugging is a very robust algorithm that is easy to implement, easy to deploy, and easy to use – provided that the underlying test case is deterministic and runs quickly enough to warrant a number of experiments.  As these are the same prerequisites that make fuzzing effective, delta debugging makes an excellent companion to fuzzing.

## Grammar-Based Input Reduction

\todo{Add}



In [ ]:
from Parser import PEGParser, parse

In [ ]:
from GrammarFuzzer import all_terminals, expansion_to_children

In [ ]:
# For logging the queue
def queue_to_string(q):
    return "[" + ", ".join([all_terminals(tree) for tree in q]) + "]"

In [ ]:
derivation_tree = ("<start>",
                   [("<expr>",
                     [("<expr>", None),
                      (" + ", []),
                         ("<term>", None)]
                     )])

In [ ]:
queue_to_string([derivation_tree, derivation_tree])

In [ ]:
from Grammars import START_SYMBOL

In [ ]:
class GrammarReducer(Reducer):
    def __init__(self, runner, grammar, start_symbol=START_SYMBOL, log=False):
        super().__init__(runner, log=log)
        self.grammar = grammar
        self.start_symbol = start_symbol
        self.parser = PEGParser(grammar, start_symbol)

In [ ]:
class GrammarReducer(GrammarReducer):
    def derivation_reductions(self, tree):
        (symbol, children) = tree
        if len(children) == 0:
            return []  # Terminal symbol

        print("Trying alternative expansions for " + symbol)

        # Possible expansions for this symbol
        expansions = self.grammar[symbol]
        print("Expansions: " + repr(expansions))

        alternatives = \
            [expansion_to_children(expansion) for expansion in expansions]

        reductions = []
        for alternative in alternatives:

            if len(alternative) > len(children):
                continue  # New alternative has more children

            match = True
            new_children_reductions = []
            # print("Trying alternative expansion " + queue_to_string(alternative))
            for alt_child in alternative:
                (alt_symbol, _) = alt_child
                child_reductions = subtrees_with_symbol(alt_symbol, tree)
                if len(child_reductions) == 0:
                    # Child not found; cannot apply rule
                    match = False
                    break

                # print("Found alternatives " + queue_to_string(child_reductions))
                new_children_reductions.append(child_reductions)

            if not match:
                continue  # Try next alternative

            # Go through the possible combinations
            for new_children in possible_combinations(new_children_reductions):
                new_tree = (symbol, new_children)

                if number_of_nodes(new_tree) >= number_of_nodes(tree):
                    continue  # No reduction

                reductions.append(new_tree)

        # Apply this recursively
        if children is not None:
            for i in range(0, len(children)):
                child = children[i]
                child_reductions = self.derivation_reductions(child)
                for reduced_child in child_reductions:
                    new_children = (children[:i] + 
                            [reduced_child] +
                            children[i + 1:])
                    reductions.append((symbol, new_children))

        # Filter duplicates
        unique_reductions = []
        for r in reductions:
            if r not in unique_reductions:
                unique_reductions.append(r)
        reductions = unique_reductions

        if len(reductions) > 0:
            # We have a new expansion
            print("Can reduce " + symbol + " " + all_terminals(tree) + \
                 " to reduced subtrees " + queue_to_string(reductions))

        return reductions

In [ ]:
class GrammarReducer(GrammarReducer):
    def reductions(self, tree):
        return self.derivation_reductions(tree)

In [ ]:
class GrammarReducer(GrammarReducer):
    # Reduce with respect to a given test
    def reduce_tree(self, tree):
        # Find possible reductions
        smallest_tree = tree
        tree_reductions = self.reductions(tree)
        print("Alternatives: " + queue_to_string(tree_reductions))

        while len(tree_reductions) > 0:
            t = tree_reductions[0]
            tree_reductions = tree_reductions[1:]
            s = all_terminals(t)
            if self.test(s) == Runner.FAIL:
                # Found new smallest tree; try to reduce that one further
                smallest_tree = t
                tree_reductions = self.reductions(t)
                tree_reductions.sort(key = lambda tree: -number_of_nodes(tree))
                print("New smallest tree: " + all_terminals(smallest_tree))

        return smallest_tree

In [ ]:
class GrammarReducer(GrammarReducer):
    def parse(self, inp):
        cursor, tree = self.parser.unify_key(self.start_symbol, inp) # TODO: Have "Parser" base class
        print(all_terminals(tree))
        return tree

    def reduce(self, inp):
        tree = self.parse(inp)
        smallest_tree = self.reduce_tree(tree)
        return all_terminals(smallest_tree)

In [ ]:
# Find all subtrees in TREE whose root is SEARCH_SYMBOL.
# If IGNORE_ROOT is true, ignore the root note of TREE.
def subtrees_with_symbol(search_symbol, tree, ignore_root = True):
    ret = []
    (symbol, children) = tree
    if not ignore_root and symbol == search_symbol:
        ret.append(tree)
    
    # Search across all children
    if children is not None:
        for c in children:
            ret += subtrees_with_symbol(search_symbol, c, False)
    return ret

In [ ]:
# convert a list [[X1, X2], [Y1, Y2], ...] 
# into [X1, Y1], [X1, Y2], [X2, Y1], [X2, Y2], ...
def possible_combinations(list_of_lists):
    if len(list_of_lists) == 0:
        return []

    # print(list_of_lists)

    ret = []
    for e in list_of_lists[0]:
        if len(list_of_lists) == 1:
            ret.append([e])
        else:
            for c in possible_combinations(list_of_lists[1:]):
                new_combo = [e] + c
                # print("New combo: ", repr(new_combo))
                ret.append(new_combo)
    return ret

In [ ]:
# Return the number of nodes
def number_of_nodes(tree):
    (symbol, children) = tree
    n = 1
    for c in children:
        n += number_of_nodes(c)
    return n

In [ ]:
from Grammars import EXPR_GRAMMAR

In [ ]:
inp = "1 + (2 * 3)"
grammar_reducer = GrammarReducer(mystery, EXPR_GRAMMAR)
grammar_reducer.reduce(inp)

## _Section 4_

\todo{Add}

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

The delta debugging algorithm discussed here stems from \cite{Zeller2002}; actually, this is the exact Python implementation as used by Zeller in 2002.  The idea of systematically reducing inputs has been discovered a number of times, although not as  automatic and generic as delta debugging. \cite{Slutz1998}, for instance, discusses systematic reduction of SQL statements for SQL databases; the general process as manual work is well described by \cite{Kernighan1999}.

\todo{Add more: hierarchical DD, grammar-based reduction, Csmith / C-reduce}


## Exercises

Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with

```markdown
**Solution.**
```

Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`).

Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution.

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_